In [1]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 7.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=5b595720ff003da28d6749958ed9acdb3df9ea89c19aff4fd4ff488ed0847e3d
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.6


In [3]:
!kaggle competitions download -c aidefensegame18011862 


  0% 0.00/171k [00:00<?, ?B/s]
100% 171k/171k [00:00<00:00, 68.1MB/s]


In [4]:
!unzip aidefensegame18011862.zip

Archive:  aidefensegame18011862.zip
  inflating: 18011862Aitrain.csv     
  inflating: alltestdata.csv         
  inflating: kaggle_18011862submission.csv  
  inflating: kaggle_18011862test.csv  


In [5]:
import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [68]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
data=pd.read_csv('18011862Aitrain.csv')
data

,연번,성별,적발횟수,나이,알콜농도,측정일시,관할경찰서
0,1,남자,1,30,0.105,2020-01-01 0:03,일산동부경찰서
1,2,남자,1,57,0.077,2020-01-01 0:13,부산연제경찰서
2,3,남자,1,68,0.089,2020-01-01 0:17,부천소사경찰서
3,4,남자,1,39,0.133,2020-01-01 0:17,마산중부경찰서
4,5,남자,1,37,측정거부,2020-01-01 0:19,수원서부경찰서
...,...,...,...,...,...,...,...
16059,16060,남자,1,40,0.164,2020-02-29 23:49,대전유성경찰서
16060,16061,남자,1,36,0.185,2020-02-29 23:49,울산남부경찰서
16061,16062,여자,1,46,0.122,2020-02-29 23:50,이천경찰서
16062,16063,남자,1,26,0.097,2020-02-29 23:51,수원남부경찰서


In [69]:
data.replace('불명',np.NAN, inplace=True)
data.dropna(inplace=True)

##경찰서 처리
data['관할경찰서'][data.관할경찰서.str.contains('서울')] = '0'
data['관할경찰서'][data.관할경찰서.str.contains('인천')] = '2'
data['관할경찰서'][data.관할경찰서.str.contains('수원')|data.관할경찰서.str.contains('일산')|data.관할경찰서.str.contains('성남')|data.관할경찰서.str.contains('용인')|data.관할경찰서.str.contains('안양')|data.관할경찰서.str.contains('안산')|data.관할경찰서.str.contains('과천')|data.관할경찰서.str.contains('광명')|data.관할경찰서.str.contains('군포')|data.관할경찰서.str.contains('부천')|
                   data.관할경찰서.str.contains('시흥')|data.관할경찰서.str.contains('김포')|data.관할경찰서.str.contains('안성')|data.관할경찰서.str.contains('오산')|data.관할경찰서.str.contains('의왕')|data.관할경찰서.str.contains('이천')|data.관할경찰서.str.contains('평택')|data.관할경찰서.str.contains('하남')|data.관할경찰서.str.contains('화성')|data.관할경찰서.str.contains('여주')|
                   data.관할경찰서.str.contains('양평')|data.관할경찰서.str.contains('고양')|data.관할경찰서.str.contains('구리')|data.관할경찰서.str.contains('남양주')|data.관할경찰서.str.contains('동두천')|data.관할경찰서.str.contains('양주')|data.관할경찰서.str.contains('의정부')|data.관할경찰서.str.contains('파주')|data.관할경찰서.str.contains('포천')|data.관할경찰서.str.contains('연천')|
                   data.관할경찰서.str.contains('가평')|data.관할경찰서.str.contains('분당')] = '1'

label1 = data['관할경찰서'] == '0'
label2 = data['관할경찰서'] == '1'
label3 = data['관할경찰서'] == '2'


pd_train = data[label1| label2| label3]
pd_train['관할경찰서'] = pd_train['관할경찰서'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_la

In [70]:
pd_train

,연번,성별,적발횟수,나이,알콜농도,측정일시,관할경찰서
0,1,남자,1,30,0.105,2020-01-01 0:03,1
2,3,남자,1,68,0.089,2020-01-01 0:17,1
4,5,남자,1,37,측정거부,2020-01-01 0:19,1
10,11,남자,1,27,0.177,2020-01-01 0:35,1
13,14,남자,1,46,0.147,2020-01-01 0:41,2
...,...,...,...,...,...,...,...
16055,16056,남자,1,49,0.072,2020-02-29 23:40,1
16058,16059,남자,1,47,0.172,2020-02-29 23:45,0
16061,16062,여자,1,46,0.122,2020-02-29 23:50,1
16062,16063,남자,1,26,0.097,2020-02-29 23:51,1


In [71]:
pd_test = pd.read_csv('alltestdata.csv')

replace_values = {'남자' : 1,'여자' : 2}

pd_train = pd_train.replace({"성별": replace_values})     ####pd_train = pd_train.replace({"성별": replace_values})
pd_test = pd_test.replace({"성별" : replace_values})

pd_train = pd_train.iloc[:,1:]

In [72]:
pd_train

,성별,적발횟수,나이,알콜농도,측정일시,관할경찰서
0,1,1,30,0.105,2020-01-01 0:03,1
2,1,1,68,0.089,2020-01-01 0:17,1
4,1,1,37,측정거부,2020-01-01 0:19,1
10,1,1,27,0.177,2020-01-01 0:35,1
13,1,1,46,0.147,2020-01-01 0:41,2
...,...,...,...,...,...,...
16055,1,1,49,0.072,2020-02-29 23:40,1
16058,1,1,47,0.172,2020-02-29 23:45,0
16061,2,1,46,0.122,2020-02-29 23:50,1
16062,1,1,26,0.097,2020-02-29 23:51,1


In [73]:
pd_train['측정일시'] = pd.to_datetime(pd_train['측정일시'], format = '%Y-%m-%d %H:%M',errors = 'raise')

pd_train['측정요일'] = pd_train['측정일시'].dt.weekday
pd_train['측정시각'] = pd_train['측정일시'].dt.hour

pd_train = pd_train[['성별','적발횟수','측정요일','측정시각','관할경찰서','나이']]


##시간 처리_test
pd_test['측정일시'] = pd.to_datetime(pd_test['측정일시'], format = '%Y-%m-%d %H:%M',errors = 'raise')

pd_test['측정요일'] = pd_test['측정일시'].dt.weekday
pd_test['측정시각'] = pd_test['측정일시'].dt.hour

pd_test = pd_test[['성별','적발횟수','측정요일','측정시각','관할경찰서']]

In [74]:
pd_train

,성별,적발횟수,측정요일,측정시각,관할경찰서,나이
0,1,1,2,0,1,30
2,1,1,2,0,1,68
4,1,1,2,0,1,37
10,1,1,2,0,1,27
13,1,1,2,0,2,46
...,...,...,...,...,...,...
16055,1,1,5,23,1,49
16058,1,1,5,23,0,47
16061,2,1,5,23,1,46
16062,1,1,5,23,1,26


In [75]:
pd_y = pd_train['나이'].astype(int)
pd_x = pd_train.iloc[:,:-1]


pd_y[pd_y<40] = 0
pd_y[pd_y>=40] = 1

pd_y.sum()

3364

In [76]:



x_train = torch.FloatTensor(np.array(pd_x)).to(device)
y_train = torch.FloatTensor(np.array(pd_y).reshape(-1,1)).to(device)

x_test = torch.FloatTensor(np.array(pd_test)).to(device)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

torch.Size([7113, 5])
torch.Size([7113, 1])
torch.Size([2719, 5])


In [77]:
# validation set 나누기

torch.manual_seed(1)
torch.cuda.manual_seed_all(1)


X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.1, random_state = 100)
print(X_train.shape)
print(X_valid.shape)
print(Y_train.shape)
print(Y_valid.shape)

torch.Size([6401, 5])
torch.Size([712, 5])
torch.Size([6401, 1])
torch.Size([712, 1])


In [78]:
x_train.shape

torch.Size([7113, 5])

In [79]:
y_train.shape

torch.Size([7113, 1])

## baseline2 code

In [402]:
lr = 1e-3
batch_size = 190    #190
Epochs = 100

drop_prob = 0.3

In [403]:
# validation data사용시!!

d = torch.utils.data.TensorDataset(X_train, Y_train)
data_loader = torch.utils.data.DataLoader(dataset = d,
                                          batch_size=batch_size,
                                          shuffle = True,
                                          drop_last=True)

In [404]:
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

linear1 = nn.Linear(5,512,bias=True)
linear2 = nn.Linear(512, 128, bias=True)
linear4 = nn.Linear(128, 128, bias=True)
linear3 = nn.Linear(128, 1, bias=True)
dropout = torch.nn.Dropout(p=drop_prob)

relu = nn.ReLU()

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear4.weight)


model = nn.Sequential(linear1, relu, 
                      linear2, relu, 
                      linear4, relu, 
                      linear3).to(device)


#for layer in model.children():
#  if isinstance(layer, nn.Linear):
#    nn.init.xavier_uniform_(layer.weight)     # xavier_uniform_



loss = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = lr)

In [405]:


total_batch = len(data_loader);
best_acc = 0
accuracy = 0

for epoch in range(Epochs+1):
  model.train()
  avg_cost = 0;

  for X, Y in data_loader:
    X = X.view(-1,5)
    optimizer.zero_grad()
    H = torch.sigmoid(model(X))
    cost=(loss(H,Y))
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    # scheduler.step(cost)
  with torch.no_grad():
    model.eval()
    # import pdb;pdb.set_trace()

    valid = torch.sigmoid(model(X_valid))
    valid[valid<0.5] = 0
    valid[valid>=0.5] = 1
    
    accuracy = accuracy_score(Y_valid.to('cpu'), valid.to('cpu').detach().numpy())*100

    if best_acc < accuracy :
      best_acc = accuracy
      print("save bestmodel, epoch: {:4d}".format(epoch))
      torch.save(model, './best_model.ptr')

  print("Epoch{:4d}/{}, cost : {:.06f}, accuracy : {:.06f}".format(epoch,
                                              Epochs,
                                              avg_cost,
                                              accuracy))

save bestmodel, epoch:    0
Epoch   0/100, cost : 0.693141, accuracy : 47.331461
Epoch   1/100, cost : 0.691521, accuracy : 47.331461
Epoch   2/100, cost : 0.690381, accuracy : 47.331461
save bestmodel, epoch:    3
Epoch   3/100, cost : 0.689116, accuracy : 50.280899
save bestmodel, epoch:    4
Epoch   4/100, cost : 0.687885, accuracy : 51.404494
save bestmodel, epoch:    5
Epoch   5/100, cost : 0.686883, accuracy : 53.792135
save bestmodel, epoch:    6
Epoch   6/100, cost : 0.686192, accuracy : 55.196629
save bestmodel, epoch:    7
Epoch   7/100, cost : 0.684695, accuracy : 55.898876
save bestmodel, epoch:    8
Epoch   8/100, cost : 0.684165, accuracy : 57.443820
save bestmodel, epoch:    9
Epoch   9/100, cost : 0.683172, accuracy : 58.286517
save bestmodel, epoch:   10
Epoch  10/100, cost : 0.682278, accuracy : 59.129213
save bestmodel, epoch:   11
Epoch  11/100, cost : 0.681676, accuracy : 59.831461
save bestmodel, epoch:   12
Epoch  12/100, cost : 0.680985, accuracy : 60.674157
sav

In [397]:
model = torch.load("./best_model.ptr")

with torch.no_grad():
  
  model.eval()
  predict = torch.sigmoid(model(x_test))
predict[predict>=0.5] = 1
predict[predict<0.5] = 0
predict.sum()

tensor(1038., device='cuda:0')

In [398]:
label = predict.to('cpu').detach().numpy()
Id = np.array([int(i) for i in range(len(predict))]).reshape(-1,1)

result = np.hstack((Id, label))
result


array([[0.000e+00, 0.000e+00],
       [1.000e+00, 0.000e+00],
       [2.000e+00, 0.000e+00],
       ...,
       [2.716e+03, 1.000e+00],
       [2.717e+03, 1.000e+00],
       [2.718e+03, 1.000e+00]])

In [399]:
df = pd.DataFrame(result, columns=(['ID','Label']))
df['ID'] = df['ID'].astype(int)
df.to_csv("submission2.csv",index=False, header=True)

In [400]:
submission2=pd.read_csv('submission2.csv')
submission2

,ID,Label
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
2714,2714,1.0
2715,2715,1.0
2716,2716,1.0
2717,2717,1.0


In [401]:
!kaggle competitions submit -c aidefensegame18011862 -f submission2.csv -m "4layer 512->256->128 마지막initx "

100% 22.8k/22.8k [00:04<00:00, 5.48kB/s]
Successfully submitted to AIDefenseGame18011862

# linear regression

In [ ]:
import torch.nn.functional as F  

nb_class = 2
nb_data = len(y_train)

W = torch.zeros((5, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)
nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693146
Epoch 1000/10000 Cost: 0.686333
Epoch 2000/10000 Cost: 0.682181
Epoch 3000/10000 Cost: 0.681770
Epoch 4000/10000 Cost: 0.705027
Epoch 5000/10000 Cost: 0.684500
Epoch 6000/10000 Cost: 0.682712
Epoch 7000/10000 Cost: 0.708847
Epoch 8000/10000 Cost: 0.710764
Epoch 9000/10000 Cost: 0.682054
Epoch 10000/10000 Cost: 0.681803


## 모델 학습

In [331]:
x_train=np.array(pd_x)
y_train=np.array(pd_y)

x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

x_train

tensor([[ 1.,  1.,  2.,  0.,  1.],
        [ 1.,  1.,  2.,  0.,  1.],
        [ 1.,  1.,  2.,  0.,  1.],
        ...,
        [ 2.,  1.,  5., 23.,  1.],
        [ 1.,  1.,  5., 23.,  1.],
        [ 2.,  1.,  5., 23.,  1.]])

In [332]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [351]:
# 학습 파라미터 설정
#learning_rate = lr=0.01
#training_epochs = 40
#batch_size = 100

lr = 1e-3
batch_size = 190
Epochs = 100

In [352]:

train = torch.utils.data.TensorDataset(x_train, y_train)

In [353]:
data_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
#xy=train

In [354]:
# 3-Layer

linear1 = torch.nn.Linear(5,256,bias=True)
linear2 = torch.nn.Linear(256,256,bias=True)
linear3 = torch.nn.Linear(256,1,bias=True)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [355]:
# Random Init => Xavier Init
torch.nn.init.xavier_normal_(linear1.weight)    # xavier_uniform_
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)

Parameter containing:
tensor([[ 0.0191,  0.2430, -0.0999,  ..., -0.1181,  0.1836, -0.0771],
        [ 0.0907, -0.0091,  0.1663,  ...,  0.0684, -0.1016,  0.0378],
        [ 0.2396, -0.0467,  0.0734,  ..., -0.1778,  0.0543,  0.1009],
        ...,
        [ 0.1676, -0.1012, -0.1013,  ...,  0.0308,  0.0260, -0.0532],
        [ 0.2252, -0.0753,  0.1286,  ...,  0.1696,  0.1550, -0.1392],
        [ 0.0278,  0.0473,  0.0826,  ...,  0.1054, -0.1051,  0.1371]],
       requires_grad=True)

In [356]:
# model = torch.nn.Sequential(linear1,relu,linear2,relu,linear4, relu, linear3).to(device)

model = torch.nn.Sequential(linear1,relu,
                            linear2, relu,
                            linear3).to(device)

In [357]:
# 손실함수와 최적화 함수

loss = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = lr) 



In [358]:

total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        # one-hot encoding되어 있지 않음
      #  X = X.view(-1,5)
        Y = Y.to(device)
        #%debug

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

RuntimeError: ignored

## 테스트

In [ ]:
pd_test

,성별,적발횟수,측정요일,측정시각,관할경찰서
0,1,1,6,0,1
1,1,1,6,0,1
2,1,1,6,0,1
3,1,1,6,0,1
4,2,1,6,0,0
...,...,...,...,...,...
2714,1,1,0,22,1
2715,1,1,0,22,1
2716,1,1,0,22,1
2717,1,1,0,22,2


In [ ]:
test=pd_test

In [ ]:
with torch.no_grad():
  x_test=test.loc[:,:]
  x_test=np.array(x_test)
  x_test=torch.from_numpy(x_test).float().to(device)

  prediction=model(x_test)
  prediction = torch.argmax(prediction, 1)

prediction

tensor([0, 0, 0,  ..., 1, 1, 1])

In [ ]:
prediction=prediction.float()
prediction

tensor([0., 0., 0.,  ..., 1., 1., 1.])

In [ ]:

submit = pd.read_csv('kaggle_18011862submission.csv')
submit

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2714,2714,0
2715,2715,0
2716,2716,0
2717,2717,0


In [ ]:
id=np.array([i for i in range(2719)]).reshape(-1,1)
prediction=prediction.reshape(-1,1)

result=np.hstack([id, prediction])
df=pd.DataFrame(result, columns=['ID','Label'])
df['ID'] = df['ID'].astype(int)
df.to_csv('submission.csv', index=False, header=True)

df


,ID,Label
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
2714,2714,1.0
2715,2715,1.0
2716,2716,1.0
2717,2717,1.0


In [ ]:
!kaggle competitions submit -c aidefensegame18011862 -f submission.csv -m "3layer 512"

100% 22.8k/22.8k [00:00<00:00, 118kB/s]
403 - Your team has used its submission allowance (20 of 20). This resets at midnight UTC (12 hours from now).


##  3layer, 128

- 학습 파라미터 설정
learning_rate = lr=0.001
training_epochs = 20
batch_size = 100



--> 0.68113